In [4]:
from couplets_utils import *
from keras.layers import *
from keras.models import Model
from keras import backend as K
import numpy as np
import tensorflow as tf

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
N_A = 128

In [6]:
sample = load_sample_datasets(vocabs_size=2000, max_len=30, batch_size=16, sample_size=1000, n_a=N_A)
sample_gen = sample['sample_gen']
index2word = sample['index2word']
word2index = sample['word2index']
vocabs_size = sample['vocabs_size']
max_len = sample['max_len']

In [7]:
print('sample_gen length :', len(sample_gen))

sample_gen length : 63


In [8]:
def create_train_model(n_x, n_a, Tx):
    input = Input(shape=(Tx, n_x), name='x0')
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
        
    a = a0
    c = c0
    
    lstm_cell = LSTM(units=n_a, return_state=True, name='lstm_0')
    dense_layer = Dense(units=n_x, activation='softmax', name='softmax_1')
        
    outputs = []
        
    for i in range(Tx):
        x = Lambda(lambda j: j[:, i, :])(input)
        x = Reshape(target_shape=(1, -1))(x)
        a, x, c = lstm_cell(x, initial_state=[a, c])
        x = dense_layer(x)
        outputs.append(x)
                            
    model = Model(inputs=[input, a0, c0], outputs=outputs)
        
    return model

In [9]:
train_model = create_train_model(vocabs_size, N_A, max_len)

In [10]:
train_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [39]:
train_model.fit_generator(sample_gen, epochs=50)

Epoch 1/50
63/63 [==============================] - 44s 703ms/step - loss: 161.2220 - softmax_1_loss: 2.6458 - softmax_1_acc: 0.0000e+00 - softmax_1_acc_1: 0.0000e+00 - softmax_1_acc_2: 0.0000e+00 - softmax_1_acc_3: 0.0000e+00 - softmax_1_acc_4: 9.9219e-04 - softmax_1_acc_5: 9.9219e-04 - softmax_1_acc_6: 0.0298 - softmax_1_acc_7: 0.0318 - softmax_1_acc_8: 0.0337 - softmax_1_acc_9: 0.0347 - softmax_1_acc_10: 0.0496 - softmax_1_acc_11: 0.0486 - softmax_1_acc_12: 0.1637 - softmax_1_acc_13: 0.1647 - softmax_1_acc_14: 0.1786 - softmax_1_acc_15: 0.1786 - softmax_1_acc_16: 0.6885 - softmax_1_acc_17: 0.6925 - softmax_1_acc_18: 0.6954 - softmax_1_acc_19: 0.6974 - softmax_1_acc_20: 0.7371 - softmax_1_acc_21: 0.7371 - softmax_1_acc_22: 0.7629 - softmax_1_acc_23: 0.7629 - softmax_1_acc_24: 0.7837 - softmax_1_acc_25: 0.7837 - softmax_1_acc_26: 0.9068 - softmax_1_acc_27: 0.9078 - softmax_1_acc_28: 0.9504 - softmax_1_acc_29: 0.9514
Epoch 2/50
63/63 [==============================] - 21s 332ms/step - 

63/63 [==============================] - 21s 336ms/step - loss: 102.6913 - softmax_1_loss: 0.0093 - softmax_1_acc: 0.0476 - softmax_1_acc_1: 0.0546 - softmax_1_acc_2: 0.0407 - softmax_1_acc_3: 0.0169 - softmax_1_acc_4: 0.1945 - softmax_1_acc_5: 0.1597 - softmax_1_acc_6: 0.0466 - softmax_1_acc_7: 0.5734 - softmax_1_acc_8: 0.0417 - softmax_1_acc_9: 0.0962 - softmax_1_acc_10: 0.0953 - softmax_1_acc_11: 0.2153 - softmax_1_acc_12: 0.3244 - softmax_1_acc_13: 0.2490 - softmax_1_acc_14: 0.2441 - softmax_1_acc_15: 0.7500 - softmax_1_acc_16: 0.7480 - softmax_1_acc_17: 0.8492 - softmax_1_acc_18: 0.7411 - softmax_1_acc_19: 0.8065 - softmax_1_acc_20: 0.7778 - softmax_1_acc_21: 0.8016 - softmax_1_acc_22: 0.8066 - softmax_1_acc_23: 0.8195 - softmax_1_acc_24: 0.8205 - softmax_1_acc_25: 0.9405 - softmax_1_acc_26: 0.9405 - softmax_1_acc_27: 0.9841 - softmax_1_acc_28: 0.9841 - softmax_1_acc_29: 1.0000
Epoch 19/50
63/63 [==============================] - 21s 338ms/step - loss: 102.1104 - softmax_1_loss: 0

63/63 [==============================] - 22s 343ms/step - loss: 91.1872 - softmax_1_loss: 0.0042 - softmax_1_acc: 0.0476 - softmax_1_acc_1: 0.0546 - softmax_1_acc_2: 0.0516 - softmax_1_acc_3: 0.0675 - softmax_1_acc_4: 0.2262 - softmax_1_acc_5: 0.2361 - softmax_1_acc_6: 0.0913 - softmax_1_acc_7: 0.6042 - softmax_1_acc_8: 0.0824 - softmax_1_acc_9: 0.1330 - softmax_1_acc_10: 0.1220 - softmax_1_acc_11: 0.2520 - softmax_1_acc_12: 0.3324 - softmax_1_acc_13: 0.2797 - softmax_1_acc_14: 0.2808 - softmax_1_acc_15: 0.7560 - softmax_1_acc_16: 0.7510 - softmax_1_acc_17: 0.8542 - softmax_1_acc_18: 0.7540 - softmax_1_acc_19: 0.8115 - softmax_1_acc_20: 0.7857 - softmax_1_acc_21: 0.8086 - softmax_1_acc_22: 0.8125 - softmax_1_acc_23: 0.8294 - softmax_1_acc_24: 0.8244 - softmax_1_acc_25: 0.9445 - softmax_1_acc_26: 0.9415 - softmax_1_acc_27: 0.9851 - softmax_1_acc_28: 0.9841 - softmax_1_acc_29: 1.0000
Epoch 37/50
63/63 [==============================] - 22s 350ms/step - loss: 90.5183 - softmax_1_loss: 0.0

In [17]:
e = train_model.evaluate_generator(sample_gen)
len(sample_gen)
list(zip(train_model.metrics_names, e))

[('loss', 228.00017150878907),
 ('softmax_1_loss', 7.600812244415283),
 ('softmax_1_loss', 7.600771919250488),
 ('softmax_1_loss', 7.601133358001709),
 ('softmax_1_loss', 7.600981288909912),
 ('softmax_1_loss', 7.600863983154297),
 ('softmax_1_loss', 7.600769088745118),
 ('softmax_1_loss', 7.60083878326416),
 ('softmax_1_loss', 7.600711822509766),
 ('softmax_1_loss', 7.600988208770752),
 ('softmax_1_loss', 7.600934204101563),
 ('softmax_1_loss', 7.601013465881348),
 ('softmax_1_loss', 7.600668468475342),
 ('softmax_1_loss', 7.600816707611084),
 ('softmax_1_loss', 7.600540649414063),
 ('softmax_1_loss', 7.600611515045166),
 ('softmax_1_loss', 7.599866081237793),
 ('softmax_1_loss', 7.60098501586914),
 ('softmax_1_loss', 7.60002645111084),
 ('softmax_1_loss', 7.599902725219726),
 ('softmax_1_loss', 7.59957597732544),
 ('softmax_1_loss', 7.599418830871582),
 ('softmax_1_loss', 7.599229839324951),
 ('softmax_1_loss', 7.599160152435303),
 ('softmax_1_loss', 7.598903335571289),
 ('softmax_1_

In [51]:
train_model.save_weights('./weights.h5')

In [9]:
def create_infer_model(n_x, n_a, Tx):
    tf.reset_default_graph()
    
    X = Input(shape=(Tx, n_x), name='input_X')
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    
    x = Lambda(lambda x: x[:, 0, :])(X)
    a = a0
    c = c0

    lstm_cell = LSTM(units=n_a, return_state=True, name='lstm_0')
    dense_layer = Dense(units=n_x, activation='softmax', name='softmax_1')
    
    def one_hot(x):
        x = K.argmax(x)
        x = tf.one_hot(x, n_x) 
        return x
    
    def select_x(x, i):
        return tf.cond(
            tf.equal(tf.reduce_sum(X[:, i+1, :]), 0),
            lambda : x,
            lambda : X[:, i+1, :]
        )
        
    def select_output(x, i):
        return tf.cond(
            tf.less(i + 1, Tx),
            lambda : select_x(x, i),
            lambda : x
        )
    
    outputs = []
    
    for i in range(Tx - 1):              
        x = Reshape(target_shape=(1, -1))(x)
        a, x, c = lstm_cell(x, initial_state=[a, c])
        x = dense_layer(x)
        x = Lambda(one_hot)(x)
        
        x = Lambda(lambda x: select_output(x, i))(x)
        outputs.append(x)
        
        
    model = Model(inputs=[X, a0, c0], outputs=outputs)
    
    return model

In [10]:
infer_model = create_infer_model(vocabs_size, N_A, max_len)

In [11]:
infer_model.load_weights('./weights.h5')

In [39]:
def write_couplets(begin_text, infer_model):
    x = convert_text_to_onehot(begin_text, vocabs_size, max_len, word2index)
    a0 = np.zeros((1, N_A))
    c0 = np.zeros((1, N_A))
    
    result = infer_model.predict([x, a0, c0])
    result_text = convert_predict_to_text(np.array(result), index2word)
    
    return result_text

In [45]:
text = '天天向上'
result_text = write_couplets(text, infer_model)
print(result_text)

天天向上，一一一山风梦；春风，万里，万里万春。      
